# Demo Notebook: Feathr Feature Store on Azure

This notebook demonstrates how Feathr Feature Store can simplify and empower your model training and inference. You will learn:

1. Define sharable features using Feathr API
2. Create a training dataset via point-in-time feature join with Feathr API
3. Materialize features to online store and then retrieve them with Feathr API

In [1]:
# Import Dependencies
import glob
import os
import tempfile
from datetime import datetime, timedelta
from math import sqrt

import pandas as pd
import pandavro as pdx
from feathr import FeathrClient
from feathr import BOOLEAN, FLOAT, INT32, ValueType
from feathr import Feature, DerivedFeature, FeatureAnchor
from feathr import BackfillTime, MaterializationSettings
from feathr import FeatureQuery, ObservationSettings
from feathr import RedisSink
from feathr import INPUT_CONTEXT, HdfsSource
from feathr import WindowAggTransformation
from feathr import TypedKey
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from azure.identity import DefaultAzureCredential
from azure.keyvault.secrets import SecretClient

## Prerequisite: Configure the required environment with Feathr Quick Start Template

In the first step (Provision cloud resources), you should have provisioned all the required cloud resources. Run the code below to install Feathr, login to Azure to get the required credentials to access more cloud resources.

**REQUIRED STEP: Fill in the resource prefix when provisioning the resources**

In [2]:
resource_prefix = "rifeathr"

Login to Azure with a device code (You will see instructions in the output):


**Permission**

To proceed with the following steps, you may need additional permission: permission to access the keyvault, permission to access the Storage Blob as a Contributor and permission to submit jobs to Synapse cluster. Skip this step if you have already given yourself the access. Otherwise, run the following lines of command in the Cloud Shell before running the cell below.

```
userId=<email_id_of_account_requesting_access>
resource_prefix=<resource_prefix>
synapse_workspace_name="${resource_prefix}syws"
keyvault_name="${resource_prefix}kv"
objectId=$(az ad user show --id $userId --query id -o tsv)
az keyvault update --name $keyvault_name --enable-rbac-authorization false
az keyvault set-policy -n $keyvault_name --secret-permissions get list --object-id $objectId
az role assignment create --assignee $userId --role "Storage Blob Data Contributor"
az synapse role assignment create --workspace-name $synapse_workspace_name --role "Synapse Contributor" --assignee $userId
```


Get all the required credentials from Azure KeyVault

In [24]:
# Get all the required credentials from Azure Key Vault
key_vault_name=resource_prefix+"kv"
synapse_workspace_url=resource_prefix+"syws"
adls_account=resource_prefix+"dls"
adls_fs_name=resource_prefix+"fs"
purview_name=resource_prefix+"purview"
key_vault_uri = f"https://{key_vault_name}.vault.azure.net"
credential = DefaultAzureCredential(exclude_interactive_browser_credential=False)
client = SecretClient(vault_url=key_vault_uri, credential=credential)
secretName = "FEATHR-ONLINE-STORE-CONN"
retrieved_secret = client.get_secret(secretName).value

# Get redis credentials; This is to parse Redis connection string.
redis_port=retrieved_secret.split(',')[0].split(":")[1]
redis_host=retrieved_secret.split(',')[0].split(":")[0]
redis_password=retrieved_secret.split(',')[1].split("password=",1)[1]
redis_ssl=retrieved_secret.split(',')[2].split("ssl=",1)[1]

# Set the resource link
os.environ['spark_config__azure_synapse__dev_url'] = f'https://{synapse_workspace_url}.dev.azuresynapse.net'
os.environ['spark_config__azure_synapse__pool_name'] = 'spark31'
os.environ['spark_config__azure_synapse__workspace_dir'] = f'abfss://{adls_fs_name}@{adls_account}.dfs.core.windows.net/fraud/feathr_project'
os.environ['online_store__redis__host'] = redis_host
os.environ['online_store__redis__port'] = redis_port
os.environ['online_store__redis__ssl_enabled'] = redis_ssl
os.environ['REDIS_PASSWORD']=redis_password
feathr_output_path = f'abfss://{adls_fs_name}@{adls_account}.dfs.core.windows.net/fraud/feathr_output'
os.environ['FEATURE_REGISTRY__API_ENDPOINT']= f'https://{resource_prefix}webapp.azurewebsites.net/api/v1'

# Define sharable features using Feathr API

In this tutorial, we use Feathr Feature Store to help create a model that predicts users product rating. To make it simple, let's just predict users' rating for ONE product for an e-commerce website. (We have an [advanced demo](./product_recommendation_demo_advanced.ipynb) that predicts ratings for arbitrary products.)


## Initialize Feathr Client

Let's initialize a Feathr client first. The Feathr client provides all the APIs we need to interact with Feathr Feature Store.

In [26]:
feathr_client = FeathrClient(config_path="feathr_config.yaml", credential=credential)

2022-11-25 18:57:04.310 | INFO     | feathr.utils._envvariableutil:get_environment_variable_with_default:51 - secrets__azure_key_vault__name not found in the config file.
2022-11-25 18:57:04.409 | INFO     | feathr.utils._envvariableutil:get_environment_variable_with_default:51 - spark_config__azure_synapse__feathr_runtime_location not found in the config file.
2022-11-25 18:57:07.170 | INFO     | feathr.utils._envvariableutil:get_environment_variable_with_default:51 - secrets__azure_key_vault__name not found in the config file.
2022-11-25 18:57:07.181 | INFO     | feathr.client:__init__:176 - Feathr Client 0.9.0 initialized successfully


## Understand the Raw Datasets
We have 3 raw datasets to work with: one observation dataset(a.k.a. label dataset) and two raw datasets to generate features.

In [9]:
# Observation dataset(a.k.a. label dataset)
# Observation dataset usually comes with a event_timestamp to denote when the observation happened.
# The label here is product_rating. Our model objective is to predict a user's rating for this product.
import pandas as pd
pd.read_csv("https://azurefeathrstorage.blob.core.windows.net/public/sample_data/product_recommendation_sample/user_observation_mock_data.csv")

,user_id,product_id,event_timestamp,product_rating
0,1,1,2021-04-01,4
1,1,2,2021-04-01,4
2,1,3,2021-04-01,4
3,1,4,2021-04-01,4
4,1,5,2021-04-01,4
5,2,1,2021-04-01,5
6,2,2,2021-04-01,5
7,2,3,2021-04-01,5
8,2,4,2021-04-01,5
9,2,5,2021-04-01,5


In [8]:
# User profile dataset
# Used to generate user features
import pandas as pd
pd.read_csv("https://azurefeathrstorage.blob.core.windows.net/public/sample_data/product_recommendation_sample/user_profile_mock_data.csv")

,user_id,gender,age,gift_card_balance,number_of_credit_cards,state,tax_rate
0,1,1,22,100,0,CA,7.5
1,2,2,17,300,1,CA,7.5
2,3,1,40,0,2,WA,7.5
3,4,1,25,100,3,WA,7.5
4,5,1,33,0,2,PA,0.0
5,6,2,19,0,2,CA,7.5
6,7,2,22,200,1,WA,7.5
7,8,2,59,300,0,PA,8.5
8,9,0,80,100,1,WA,8.5
9,10,0,39,100,0,WA,7.5


In [9]:
# User purchase history dataset.
# Used to generate user features. This is activity type data, so we need to use aggregation to genearte features.
import pandas as pd
pd.read_csv("https://azurefeathrstorage.blob.core.windows.net/public/sample_data/product_recommendation_sample/user_purchase_history_mock_data.csv")

,user_id,purchase_date,purchase_amount
0,1,2021-01-01,0.33
1,1,2021-03-03,574.35
2,1,2021-01-03,796.07
3,2,2021-01-04,342.15
4,2,2021-03-05,280.46
5,2,2021-01-06,664.18
6,3,2021-01-07,359.02
7,3,2021-01-08,357.12
8,3,2021-01-09,845.40
9,4,2021-01-10,103.92


 After a bit of data exploration, we want to create a training dataset like this:

 
![Feature Flow](https://github.com/feathr-ai/feathr/blob/main/docs/images/product_recommendation.jpg?raw=true)

## What's a Feature in Feathr
A feature is an individual measurable property or characteristic of a phenomenon which is sometimes time-sensitive. 

In Feathr, feature can be defined by the following characteristics:
1. The typed key (a.k.a. entity id): identifies the subject of feature, e.g. a user id of 123, a product id of SKU234456.
2. The feature name: the unique identifier of the feature, e.g. user_age, total_spending_in_30_days.
3. The feature value: the actual value of that aspect at a particular time, e.g. the feature value of the person's age is 30 at year 2022.

You can feel that this is defined from a feature consumer(a person who wants to use a feature) perspective. It only tells us what a feature is like. In later sections, you can see how a feature consumer can access the features in a very simple way.

To define a feature as well as how it can be produced, additionally we need:
1. Feature source: what source data that this feature is based on
2. Transformation: what transformation is used to transform the source data into feature. Transformation can be optional when you just want to take a column out from the source data.

(For more details on feature definition, please refer to the [Feathr Feature Definition Guide](https://github.com/feathr-ai/feathr/blob/main/docs/concepts/feature-definition.md))

### Define Sources Section with Preprocssing
A [feature source](https://feathr.readthedocs.io/en/latest/#feathr.Source) defines where to find the source data and how to use the source data for the upcoming feature transformation. There are different types of feature sources that you can use. HdfsSource is the most commonly used one that can connect you to data lake, Snowflake database tables etc. It's simliar to database connector.

To define HdfsSource, we need:
1. `name`: It's used for you to recognize it. It has to be unique among all other feature source. Here we use `userProfileData`. 
2. `path`: It points to the location that we can find the source data.
3. `preprocessing`(optional): If you want some preprocessing other than provided transformation, you can do it here. This preprocessing will be applied all the transformations of this source.
4. `event_timestamp_column`(optioanl): there are `event_timestamp_column` and `timestamp_format` used for point-in-time join and we will cover them later.

See [the python API documentation](https://feathr.readthedocs.io/en/latest/#feathr.HdfsSource) to get the details of each input fields. 

In [27]:
from pyspark.sql import SparkSession, DataFrame

def feathr_udf_preprocessing(df: DataFrame) -> DataFrame:
    from pyspark.sql.functions import col
    df = df.withColumn("tax_rate_decimal", col("tax_rate")/100)
    df.show(10)
    return df

batch_source = HdfsSource(name="userProfileData",
                          path="wasbs://public@azurefeathrstorage.blob.core.windows.net/sample_data/product_recommendation_sample/user_profile_mock_data.csv",
                          preprocessing=feathr_udf_preprocessing)

### Define Features on Top of Data Sources
To define features on top of the `HdfsSource`, we need to:
1. specify the key of this feature: feature are like other data, they are keyed by some id. For example, user_id, product_id. You can also define compound keys.
2. specify the name of the feature via `name` parameter and how to transform it from source data via `transform` parameter. Also some other metadata, like `feature_type`.
3. group them together so we know it's from one `HdfsSource` via `FeatureAnchor`. Also give it a unique name via `name` parameter so we can recognize it.

It's called FeatureAnchor since it's like this group of features are anchored to the source. There are other types of features that are computed on top of other features(a.k.a. derived feature which we will cover in next section)

In [28]:
user_id = TypedKey(key_column="user_id",
                   key_column_type=ValueType.INT32,
                   description="user id",
                   full_name="product_recommendation.user_id")

feature_user_age = Feature(name="feature_user_age",
                           key=user_id,
                           feature_type=INT32, 
                           transform="age")
feature_user_tax_rate = Feature(name="feature_user_tax_rate",
                                key=user_id,
                                feature_type=FLOAT,
                                transform="tax_rate_decimal")
feature_user_gift_card_balance = Feature(name="feature_user_gift_card_balance",
                                    key=user_id,
                                    feature_type=FLOAT,
                                    transform="gift_card_balance")
feature_user_has_valid_credit_card = Feature(name="feature_user_has_valid_credit_card",
                                    key=user_id,
                                    feature_type=BOOLEAN,
                                    transform="number_of_credit_cards > 0")
                                    
features = [
    feature_user_age,
    feature_user_tax_rate,
    feature_user_gift_card_balance,
    feature_user_has_valid_credit_card
]

request_anchor = FeatureAnchor(name="anchored_features",
                               source=batch_source,
                               features=features)

### Window aggregation features

Using [window aggregations](https://en.wikipedia.org/wiki/Window_function_%28SQL%29) can help us create more powerful features. A window aggregation feature compress large amount of information into one single feature value. Using our raw data as an example, we have the users' purchase history data that might be quite some rows, we want to create a window aggregation feature that represents their last 90 days of average purcahse amount.

Feathr provides a nice API to help us create such window aggregation features.

To create this window aggregation feature via Feathr, we just need to define the following parameters with `WindowAggTransformation` API:
1. `agg_expr`: the field/column you want to aggregate. It can be a ANSI SQL expression. So we just write `cast_float(purchase_amount)`(the raw data might be in string form, let's cast_float).
2. `agg_func`: the aggregation function you want. We want to use `AVG` here.
3. `window`: the aggregation window size you want. Let's use `90d`. You can tune your windows to create different window aggregation features.

For window aggregation functions, see the supported fields below:

| Aggregation Type | Input Type | Description |
| --- | --- | --- |
|SUM, COUNT, MAX, MIN, AVG	|Numeric|Applies the the numerical operation on the numeric inputs. |
|MAX_POOLING, MIN_POOLING, AVG_POOLING	| Numeric Vector | Applies the max/min/avg operation on a per entry bassis for a given a collection of numbers.|
|LATEST| Any |Returns the latest not-null values from within the defined time window |

(Note that the `agg_func` should be any of these.)

After you have defined features and sources, bring them together to build an anchor:

In [29]:
purchase_history_data = HdfsSource(name="purchase_history_data",
                          path="wasbs://public@azurefeathrstorage.blob.core.windows.net/sample_data/product_recommendation_sample/user_purchase_history_mock_data.csv",
                          event_timestamp_column="purchase_date",
                          timestamp_format="yyyy-MM-dd")
                          
agg_features = [Feature(name="feature_user_total_purchase_in_90days",
                        key=user_id,
                        feature_type=FLOAT,
                        transform=WindowAggTransformation(agg_expr="cast_float(purchase_amount)",
                                                          agg_func="AVG",
                                                          window="90d"))
                ]

agg_anchor = FeatureAnchor(name="aggregationFeatures",
                           source=purchase_history_data,
                           features=agg_features)

### Derived Features Section
Derived features are features that are computed from other Feathr features. They could be computed from anchored features, or other derived features.

Typical usage includes feature cross(f1 * f2), or computing cosine similarity between two features.

The syntax works in a similar way.

In [30]:
feature_user_purchasing_power = DerivedFeature(name="feature_user_purchasing_power",
                                      key=user_id,
                                      feature_type=FLOAT,
                                      input_features=[feature_user_gift_card_balance, feature_user_has_valid_credit_card],
                                      transform="feature_user_gift_card_balance + if_else(toBoolean(feature_user_has_valid_credit_card), 100, 0)")

### Build Features
Lastly, we need to build those features so that it can be consumed later. Note that we have to build both the "anchor" and the "derived" features.

In [31]:
feathr_client.build_features(anchor_list=[agg_anchor, request_anchor], 
                      derived_feature_list=[feature_user_purchasing_power])

### Optional: A Special Type of Feature: Request Feature
For advanced user cases, in some cases, features defined on top of request data(a.k.a. observation data) may have no entity key or timestamp.
It is merely a function/transformation executing against request data at runtime.
For example, the day of week of the request, which is calculated by converting the request UNIX timestamp.
In this case, the `source` section should be `INPUT_CONTEXT` to indicate the source of those defined anchors.

We won't cover the details it in this notebook.

## Create training data using point-in-time correct feature join

A training dataset usually contains entity id column(s), multiple feature columns, event timestamp column and label/target column. 

To create a training dataset using Feathr, we need to provide a feature join settings to specify
what features and how these features should be joined to the observation data. 

(To learn more on this topic, please refer to [Point-in-time Correctness](https://github.com/feathr-ai/feathr/blob/main/docs/concepts/point-in-time-join.md))

In [15]:
# Synapse and Databricks have different output path format
if feathr_client.spark_runtime == 'databricks':
    output_path = 'dbfs:/feathrazure_test.avro'
else:
    output_path = feathr_output_path

# Features that we want to request
feature_query = FeatureQuery(feature_list=["feature_user_age", 
                                           "feature_user_tax_rate", 
                                           "feature_user_gift_card_balance", 
                                           "feature_user_has_valid_credit_card", 
                                           "feature_user_total_purchase_in_90days",
                                           "feature_user_purchasing_power"], 
                             key=user_id)
settings = ObservationSettings(
    observation_path="wasbs://public@azurefeathrstorage.blob.core.windows.net/sample_data/product_recommendation_sample/user_observation_mock_data.csv",
    event_timestamp_column="event_timestamp",
    timestamp_format="yyyy-MM-dd")
feathr_client.get_offline_features(observation_settings=settings,
                            feature_query=feature_query,
                            output_path=output_path)
feathr_client.wait_job_to_finish(timeout_sec=50000000)

2022-10-23 21:02:19.881 | INFO     | feathr.spark_provider._synapse_submission:upload_or_get_cloud_path:66 - Uploading /tmp/tmpikx4ix11/feathr_pyspark_driver.py to cloud..
2022-10-23 21:02:19.882 | INFO     | feathr.spark_provider._synapse_submission:upload_file:409 - Uploading file feathr_pyspark_driver.py
2022-10-23 21:02:20.059 | INFO     | feathr.spark_provider._synapse_submission:upload_file:415 - /tmp/tmpikx4ix11/feathr_pyspark_driver.py is uploaded to location: abfss://mufyfeathrfs@mufyfeathrdls.dfs.core.windows.net/feathr_project/feathr_pyspark_driver.py
2022-10-23 21:02:20.061 | INFO     | feathr.spark_provider._synapse_submission:upload_or_get_cloud_path:70 - /tmp/tmpikx4ix11/feathr_pyspark_driver.py is uploaded to location: abfss://mufyfeathrfs@mufyfeathrdls.dfs.core.windows.net/feathr_project/feathr_pyspark_driver.py
2022-10-23 21:02:20.076 | INFO     | feathr.spark_provider._synapse_submission:upload_or_get_cloud_path:66 - Uploading /tmp/tmpikx4ix11/feature_join_conf/featu

## Download the result and show the result

Let's use the helper function `get_result_df` to download the result and view it:

In [16]:
def get_result_df(client: FeathrClient) -> pd.DataFrame:
    """Download the job result dataset from cloud as a Pandas dataframe."""
    res_url = client.get_job_result_uri(block=True, timeout_sec=600)
    tmp_dir = tempfile.TemporaryDirectory()
    client.feathr_spark_launcher.download_result(result_path=res_url, local_folder=tmp_dir.name)
    dataframe_list = []
    # assuming the result are in avro format
    for file in glob.glob(os.path.join(tmp_dir.name, '*.avro')):
        dataframe_list.append(pdx.read_avro(file))
    vertical_concat_df = pd.concat(dataframe_list, axis=0)
    tmp_dir.cleanup()
    return vertical_concat_df

df_res = get_result_df(feathr_client)

df_res

2022-10-23 21:07:53.653 | INFO     | feathr.spark_provider._synapse_submission:wait_for_completion:176 - Current Spark job status: success
2022-10-23 21:07:53.820 | INFO     | feathr.spark_provider._synapse_submission:download_file:427 - Beginning reading of results from abfss://mufyfeathrfs@mufyfeathrdls.dfs.core.windows.net/feathr_output
2022-10-23 21:07:54.744 | INFO     | feathr.spark_provider._synapse_submission:download_file:456 - Finish downloading files from abfss://mufyfeathrfs@mufyfeathrdls.dfs.core.windows.net/feathr_output to /tmp/tmp_0art4d6.


,user_id,product_id,event_timestamp,product_rating,feature_user_total_purchase_in_90days,feature_user_gift_card_balance,feature_user_has_valid_credit_card,feature_user_tax_rate,feature_user_age,feature_user_purchasing_power
0,9,1,2021-04-01,5,329.216675,100.0,True,0.085,80,200.0
1,9,2,2021-04-01,5,329.216675,100.0,True,0.085,80,200.0
2,9,3,2021-04-01,5,329.216675,100.0,True,0.085,80,200.0
3,9,4,2021-04-01,5,329.216675,100.0,True,0.085,80,200.0
0,1,1,2021-04-01,4,456.916656,100.0,False,0.075,22,100.0
1,1,2,2021-04-01,4,456.916656,100.0,False,0.075,22,100.0
2,1,3,2021-04-01,4,456.916656,100.0,False,0.075,22,100.0
3,1,4,2021-04-01,4,456.916656,100.0,False,0.075,22,100.0
4,1,5,2021-04-01,4,456.916656,100.0,False,0.075,22,100.0
0,6,1,2021-04-01,2,546.703369,0.0,True,0.075,19,100.0


## Train a machine learning model
After getting all the features, let's train a machine learning model with the converted feature by Feathr:

In [25]:
# drop non-feature columns
from sklearn.ensemble import GradientBoostingRegressor
final_df = df_res
final_df.drop(["event_timestamp"], axis=1, inplace=True, errors='ignore')
final_df.fillna(0, inplace=True)
final_df['product_rating'] = final_df['product_rating'].astype("float64")

train_x, test_x, train_y, test_y = train_test_split(final_df.drop(["product_rating", "user_id", "product_id"], axis=1),
                                                    final_df["product_rating"],
                                                    test_size=0.2,
                                                    random_state=42)


model = GradientBoostingRegressor()
model.fit(train_x, train_y)

y_predict = model.predict(test_x)

y_actual = test_y.values.flatten().tolist()
rmse = sqrt(mean_squared_error(y_actual, y_predict))

sum_actuals = sum_errors = 0

for actual_val, predict_val in zip(y_actual, y_predict):
    abs_error = actual_val - predict_val
    if abs_error < 0:
        abs_error = abs_error * -1

    sum_errors = sum_errors + abs_error
    sum_actuals = sum_actuals + actual_val

mean_abs_percent_error = sum_errors / sum_actuals
print("Model MAPE:")
print(mean_abs_percent_error)
print()
print("Model Accuracy:")
print(1 - mean_abs_percent_error)


Model MAPE:
8.230754469954842e-06

Model Accuracy:
0.99999176924553


In [58]:
train_x.shape

print (type (train_x))

print (train_x.head(4))

#train_x.columns




<class 'pandas.core.frame.DataFrame'>
   feature_user_total_purchase_in_90days  feature_user_gift_card_balance  \
1                             490.896637                           100.0   
1                             441.213348                           200.0   
0                             546.703369                             0.0   
0                             329.216675                           100.0   

   feature_user_has_valid_credit_card  feature_user_tax_rate  \
1                                True                  0.075   
1                                True                  0.075   
0                                True                  0.075   
0                                True                  0.085   

   feature_user_age  feature_user_purchasing_power  
1                25                          200.0  
1                22                          300.0  
0                19                          100.0  
0                80                          200

## Materialize feature value into offline/online storage

In the previous section, we demonstrated how Feathr can compute feature value to generate training dataset from feature definition on-they-fly.

Now let's talk about how we can use the trained models. We can use the trained models for offline inference as well as online inference. In both cases, we need features to be feed into the models. For offline inference, you can compute and get the features on-demand; or you can store the computed features to some offline database for later offline inference.

For online inference, we can use Feathr to compute and store the features in the online database. Then use it for online inference when the request comes.

![img](../images/online_inference.jpg)


In this section, we will focus on materialize features to online store. For materialization to offline store, you can check out our [user guide](https://github.com/feathr-ai/feathr/blob/main/docs/concepts/materializing-features.md#materializing-features-to-offline-store).

We can push the computed features to the online store like below:

In [28]:
backfill_time = BackfillTime(start=datetime(2020, 5, 20), 
                             end=datetime(2020, 5, 20), 
                             step=timedelta(days=1))
redisSink = RedisSink(table_name="productRecommendationDemoFeature")
settings = MaterializationSettings(name="productRecommendationFeatureSetting",
                                   backfill_time=backfill_time,
                                   sinks=[redisSink],
                                   feature_names=['feature_user_total_purchase_in_90days',
       'feature_user_gift_card_balance', 'feature_user_has_valid_credit_card',
       'feature_user_tax_rate', 'feature_user_age',
       'feature_user_purchasing_power'])


feathr_client.materialize_features(settings)
feathr_client.wait_job_to_finish(timeout_sec=500)

2022-10-23 21:18:26.594 | INFO     | feathr.spark_provider._synapse_submission:upload_or_get_cloud_path:66 - Uploading /tmp/tmpikx4ix11/feathr_pyspark_driver.py to cloud..
2022-10-23 21:18:26.595 | INFO     | feathr.spark_provider._synapse_submission:upload_file:409 - Uploading file feathr_pyspark_driver.py
2022-10-23 21:18:27.119 | INFO     | feathr.spark_provider._synapse_submission:upload_file:415 - /tmp/tmpikx4ix11/feathr_pyspark_driver.py is uploaded to location: abfss://mufyfeathrfs@mufyfeathrdls.dfs.core.windows.net/feathr_project/feathr_pyspark_driver.py
2022-10-23 21:18:27.120 | INFO     | feathr.spark_provider._synapse_submission:upload_or_get_cloud_path:70 - /tmp/tmpikx4ix11/feathr_pyspark_driver.py is uploaded to location: abfss://mufyfeathrfs@mufyfeathrdls.dfs.core.windows.net/feathr_project/feathr_pyspark_driver.py
2022-10-23 21:18:27.120 | INFO     | feathr.utils._envvariableutil:get_environment_variable:82 - KAFKA_SASL_JAAS_CONFIG is not set in the environment variables

## Fetch feature value from online store
We can then get the features from the online store (Redis) via the client's `get_online_features` or `multi_get_online_features` API.

In [82]:
import numpy as np
import pandas as pd

single_online_inference_data = feathr_client.get_online_features('productRecommendationDemoFeature', 
                           '2', ['feature_user_total_purchase_in_90days',
       'feature_user_gift_card_balance', 'feature_user_has_valid_credit_card',
       'feature_user_tax_rate', 'feature_user_age',
       'feature_user_purchasing_power'])


print (type(single_online_inference_data))

single_online_inference_data_np  = np.array(single_online_inference_data)

single_online_inference_data_np= single_online_inference_data_np.reshape (-1, 6)


<class 'list'>


In [99]:
import numpy as np
import pandas as pd
multiple_online_inference_data = feathr_client.multi_get_online_features('productRecommendationDemoFeature', 
                                 ['1', '2'], 
                                 ['feature_user_total_purchase_in_90days',
       'feature_user_gift_card_balance', 'feature_user_has_valid_credit_card',
       'feature_user_tax_rate', 'feature_user_age',
       'feature_user_purchasing_power'])

df = pd.DataFrame(multiple_online_inference_data)
df = df.values.reshape (-1, 6)


(2, 6)

In [100]:
model.predict (df)

/anaconda/envs/feathr-env2/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(


array([4.99996655, 4.99996655])

### Registering and Fetching features

We can also register the features with an Apache Atlas compatible service, such as Azure Purview, and share the registered features across teams:

In [32]:
feathr_client.register_features()


RuntimeError: Failed to call registry API, status is 400, error is <html>
<head><title>400 Request Header Or Cookie Too Large</title></head>
<body>
<center><h1>400 Bad Request</h1></center>
<center>Request Header Or Cookie Too Large</center>
<hr><center>nginx/1.18.0</center>
</body>
</html>


In [103]:
feathr_client.list_registered_features(project_name="feathr_getting_started_demo2")

[{'name': 'feature_user_purchasing_power',
  'id': '26963c3f-25cd-48ec-bf76-c1dcb3662107',
  'qualifiedName': 'feathr_getting_started_demo2__feature_user_purchasing_power'},
 {'name': 'feature_user_gift_card_balance',
  'id': '508b60db-c7f9-4154-9fa1-31478d1366ef',
  'qualifiedName': 'feathr_getting_started_demo2__anchored_features__feature_user_gift_card_balance'},
 {'name': 'feature_user_tax_rate',
  'id': '942de3ae-b26e-4748-a96c-c4a7fc10f4d0',
  'qualifiedName': 'feathr_getting_started_demo2__anchored_features__feature_user_tax_rate'},
 {'name': 'feature_user_total_purchase_in_90days',
  'id': 'c892d722-9295-4273-995d-ab4dcc375503',
  'qualifiedName': 'feathr_getting_started_demo2__aggregationFeatures__feature_user_total_purchase_in_90days'},
 {'name': 'feature_user_has_valid_credit_card',
  'id': 'ebbb6abf-bb1b-4784-b819-92b01bbfa374',
  'qualifiedName': 'feathr_getting_started_demo2__anchored_features__feature_user_has_valid_credit_card'},
 {'name': 'feature_user_age',
  'id': 'f